In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1623317180924_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
input_data = "s3a://udacity-dend/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#from pyspark import StructType, StructField
from pyspark.sql.types import StructType, StructField, IntegerType,StringType,  DoubleType,LongType
schema = StructType([
      StructField("num_songs",IntegerType(),True),
      StructField("artist_id",StringType(),True),
      StructField("artist_latitude",DoubleType(),True),
      StructField("artist_longitude",DoubleType(),True),
      StructField("artist_location",StringType(),True),
      StructField("artist_name",StringType(),True),
      StructField("song_id",StringType(),True),
      StructField("title",StringType(),True),
      StructField("duration",DoubleType(),True),
      StructField("year",IntegerType(),True)
  ])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
input_song_data=input_data+"song_data"
df = spark.read.schema(schema).json(input_song_data).drop_duplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
output_data = "s3://aws-sparkify-datalake"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.createOrReplaceTempView("songs_table")
columns = ['song_id', 'title', 'artist_id', 'year', 'duration']

songs_table = spark.sql(
    """
    SELECT song_id, title, artist_id, year, duration
    FROM songs_table
    """
    ).toDF(*columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
songs_table.write.partitionBy("year", "artist_id").parquet(os.path.join(output_data, "songs.parquet"), "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.createOrReplaceTempView("artists_table")
columns=["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]
artists_table = spark.sql(
    """
    SELECT song_id, title, artist_id, year, duration
    FROM artists_table
    """
    ).toDF(*columns)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
artists_table.write.parquet(os.path.join(output_data, "artists.parquet"), "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
schema_1 = StructType([
      StructField("artist",StringType(),True),
      StructField("auth",StringType(),True),
      StructField("firstName",StringType(),True),
      StructField("gender",StringType(),True),
      StructField("itemInSession",IntegerType(),True),
      StructField("lastName",StringType(),True),
      StructField("length",DoubleType(),True),
      StructField("level",StringType(),True),
      StructField("location",StringType(),True),
      StructField("method",StringType(),True),
    StructField("page",StringType(),True),
      StructField("registration",LongType(),True),
      StructField("sessionId",IntegerType(),True),
      StructField("song",StringType(),True),
      StructField("status",IntegerType(),True),
      StructField("ts",StringType(),True),
    StructField("userAgent",StringType(),True),
      StructField("userId",IntegerType(),True)
    
  ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
from pyspark.sql.functions import col
input_song_data=input_data+"log_data"
df_log = spark.read.schema(schema_1).json(input_song_data).drop_duplicates().filter(col('page')=="NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_log.createOrReplaceTempView("users_table")
columns = ['userId', 'firstName', 'lastName', 'gender', 'level']

    # write users table to parquet files
users_table = spark.sql("""
    SELECT userId, firstName, lastName, gender, level
    FROM users_table
    """).toDF(*columns)

users_table.write.parquet(os.path.join(output_data, "users.parquet"), "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df=df_log
get_start_time = udf(lambda x: datetime.fromtimestamp(x / 1000.0).strftime('%Y-%m-%d %H:%M:%S'))
get_hour = udf(lambda x: datetime.fromtimestamp(x / 1000.0).hour)
get_day = udf(lambda x: datetime.fromtimestamp(x / 1000.0).day)
get_week = udf(lambda x: datetime.fromtimestamp(x / 1000.0).strftime('%W'))
get_month = udf(lambda x: datetime.fromtimestamp(x / 1000.0).month)
get_year = udf(lambda x: datetime.fromtimestamp(x / 1000.0).year)
get_weekday = udf(lambda x: datetime.fromtimestamp(x / 1000.0).strftime('%A'))

df = df.withColumn('start_time', get_start_time(df['ts']))
df = df.withColumn('hour', get_hour(df['ts']))
df = df.withColumn('day', get_day(df['ts']))
df = df.withColumn('week', get_week(df['ts']))
df = df.withColumn('month', get_month(df['ts']))
df = df.withColumn('year', get_year(df['ts']))
df = df.withColumn('week_day', get_weekday(df['ts']))

df.createOrReplaceTempView("time_table")
df.printSchema()

columns = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'week_day']

    # extract columns to create time table
time_table = spark.sql(
"""
    SELECT start_time, hour, day, week, month, year, week_day
    FROM time_table
""").toDF(*columns)

    # write time table to parquet files partitioned by year and month
time_table.write.partitionBy("year", "month").parquet(os.path.join(output_data, "time.parquet"), "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: integer (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: integer (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- start_time: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- week_day: string (nullable = true)

In [15]:
import pyspark.sql.functions as F
df = df.withColumn('songplay_id', F.monotonically_increasing_id())
df.createOrReplaceTempView("songplays_table")

song_data = input_song_data
song_df = spark.read.schema(schema).json(song_data).drop_duplicates()
song_df.createOrReplaceTempView("songs_table")

columns = ['songplay_id', 'start_time', 'userId', 'level', 'sessionId', 'location', 'userAgent', 'year', 'month',
               'length', 'song_id', 'artist_id', 'title', 'artist_name', 'duration']

songplays_table = spark.sql(
        """
            SELECT sp.songplay_id, sp.start_time, sp.userId, sp.level, sp.sessionId, sp.location, sp.userAgent, sp.year, 
            sp.month, sp.length, s.song_id, s.artist_id, s.title, s.artist_name, s.duration
            FROM songplays_table AS sp 
            JOIN songs_table AS s ON sp.song = s.title AND sp.artist = s.artist_name AND sp.length = s.duration
        """).toDF(*columns)

songplays_table.write.partitionBy("year", "month").parquet(os.path.join(output_data, "songplays.parquet"), "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…